In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import LabelEncoder


In [20]:
# load data final-dataset-short.csv
df = pd.read_csv('final-dataset-short.csv', index_col=0)
df.head()
print(df.shape)
df.info()
# change all columns with Dtype Object to numerical data
le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])

df.info()






(25, 27)
<class 'pandas.core.frame.DataFrame'>
Float64Index: 25 entries, 3.0 to 14.0
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DES_ADD           25 non-null     float64
 1   PKT_ID            25 non-null     int64  
 2   FROM_NODE         25 non-null     int64  
 3   TO_NODE           25 non-null     int64  
 4   PKT_TYPE          25 non-null     object 
 5   PKT_SIZE          25 non-null     int64  
 6   FLAGS             25 non-null     object 
 7   FID               25 non-null     int64  
 8   SEQ_NUMBER        25 non-null     int64  
 9   NUMBER_OF_PKT     25 non-null     int64  
 10  NUMBER_OF_BYTE    25 non-null     int64  
 11  NODE_NAME_FROM    25 non-null     object 
 12  NODE_NAME_TO      25 non-null     object 
 13  PKT_IN            25 non-null     float64
 14  PKT_OUT           25 non-null     float64
 15  PKT_R             25 non-null     float64
 16  PKT_DELAY_NODE    25 non-null  

In [21]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# encode the PKT_CLASS column
le = LabelEncoder()
y = le.fit_transform(df['PKT_CLASS'])

# make a model to predict pkt_class
# 1. create a model
model = Sequential()
model.add(LSTM(128, input_shape=(df.shape[1]-1, 1), activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

# 2. compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 3. convert the input data to float32 and fit the model
with tf.device('/GPU:0'):
    x = df.drop('PKT_CLASS', axis=1).values.astype(np.float32)
    x = x.reshape(-1, df.shape[1]-1, 1)
    model.fit(x, y, epochs=3, validation_split=0.3)


Epoch 1/3
1/1 [==============================] - 3s 3s/step - loss: nan - accuracy: 0.2353 - val_loss: nan - val_accuracy: 0.6250
Epoch 2/3
1/1 [==============================] - 0s 131ms/step - loss: nan - accuracy: 0.8824 - val_loss: nan - val_accuracy: 0.6250
Epoch 3/3
1/1 [==============================] - 0s 116ms/step - loss: nan - accuracy: 0.8824 - val_loss: nan - val_accuracy: 0.6250
